# RAG using LangChain

### Крок 1: Завантаження бази знань

Використовуємо бібліотеку `datasets` від Hugging Face для завантаження даних. Цей крок дозволяє нам отримати доступ до специфічної бази даних, яка вже поділена на частини для навчання та тестування. Дані містять текст документів, які будемо використовувати для подальшої обробки.

In [9]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

In [10]:
dataset_name = "databricks/databricks-dolly-15k"
page_content_column = "context"
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)

In [11]:
data = loader.load()
data[:2]

README.md:   0%|          | 0.00/8.20k [00:00<?, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

[Document(metadata={'instruction': 'When did Virgin Australia start operating?', 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}, page_content='"Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia\'s domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney."'),
 Document(metadata={'instruction': 'Which is a species of fish? Tope or Rope', 'response': 'Tope', 'category': 'classification'}, page_content='""')]

### Крок 2: Розділення документів

Для ефективного вилучення інформації та зручності у подальшому пошуку, документи потрібно розділити на менші частини. Це дозволяє системі швидше знаходити відповідні фрагменти тексту, які відповідають запитам користувача. Використовуємо `RecursiveCharacterTextSplitter` для членування текстів на сегменти, що забезпечує кращу маневреність та точність при пошуку відповідей на запити.

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(data)

### Крок 3: Створення векторної бази даних та налаштування вкладень
Після розділення тексту на чанки, кожен чанк перетворюється на вектор за допомогою моделі вкладень. Ці вектори зберігаються в базі даних FAISS для швидкого векторного пошуку.

In [14]:
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
db = FAISS.from_documents(docs, embeddings)

### Крок 4: Встановлення системи вилучення інформації та генерація відповідей
Останній крок - налаштування системи для вилучення інформації та генерації текстових відповідей на запити користувача. Використовуючи модель для генерації тексту, система аналізує контекст, вилучений з бази даних, і формулює відповідь.

In [21]:
import warnings
import torch
from transformers import pipeline, AutoTokenizer
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

warnings.filterwarnings("ignore", category=FutureWarning)
device = 0 if torch.cuda.is_available() else -1 

question_answerer = pipeline(
    "question-answering",
    model="Intel/dynamic_tinybert",
    tokenizer=AutoTokenizer.from_pretrained("Intel/dynamic_tinybert"),
    device=device
)

llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.7, "max_length": 512},
)

retriever = db.as_retriever(search_kwargs={"k": 4})
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

In [22]:
question = "What is the capital city of the US"
qa_inputs = {
    "question": question,
    "context": retriever.get_relevant_documents(question)[0].page_content
}

result = question_answerer(qa_inputs)
print(result['answer'])

New York City
